In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.interpolate import griddata

c:\Users\adrim\.conda\envs\TFG\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\adrim\.conda\envs\TFG\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\adrim\.conda\envs\TFG\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\Users\adrim\.conda\envs\TFG\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Análisis del experimento 2: Detección del estado del ojo sobre vídeos (DATASET EYEBLINK8)

In [47]:
experiment2_df = (
    pd.read_csv("experiment2_baseline.csv").fillna(0)
)
experiment2_df = experiment2_df[experiment2_df["Unnamed: 0"] != "total"]
experiment2_df.head()

,Unnamed: 0,num_real_blinks,num_method_blinks,num_real_closed_eyes,num_method_closed_eyes,num_hit_closed_eyes,true_positives,false_positives,false_negatives,accuracy,precision,recall,f1-score,method,threshold
1,1,38,1,315,3,3,1,0,37,0.026316,1.0,0.026316,0.051282,1,0.0
2,10,72,9,574,12,12,9,0,63,0.125000,1.0,0.125000,0.222222,1,0.0
3,11,43,3,345,6,6,3,0,40,0.069767,1.0,0.069767,0.130435,1,0.0
4,2,88,3,1024,6,6,3,0,85,0.034091,1.0,0.034091,0.065934,1,0.0
5,3,65,10,473,15,15,9,1,56,0.138462,0.9,0.138462,0.240000,1,0.0


In [45]:
method_list = list(experiment2_df["method"].unique())
method_df_list = []
for method in method_list:
    method_df_list.append(experiment2_df[experiment2_df["method"] == method])

In [46]:
plot = go.Figure()

color_dict = {
    1: "#003f5c",
    2: "#bc5090",
    3: "#ffa600",
}

for ind, df in enumerate(method_df_list): 
    plot.add_traces([
        go.Scatter(x=df["threshold"], y=df["f1-score"], mode="lines+markers", name=f"Método {ind+1}", marker_color=color_dict[ind+1])
    ])
plot.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='rgb(24, 24, 24)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
        title={
            "text": "<b>Threshold</b>",
            'font': {
                'size': 14,
            }
        },
        dtick=0.05,
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        linecolor='rgb(24, 24, 24)',
        linewidth=2,
        title={
            "text": "<b>F1-score</b>",
            'font': {
                'size': 14,
            }
        }
    ),
    title={
        'text': "Evolución de la f1-score según el valor del threshold estático",
        'font': {
            'size': 18,
        },
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'bordercolor': '#000000',
        'borderwidth': 0,
    },
    width=1000,
    height=500,
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    template="plotly_white",
    showlegend=True,
    plot_bgcolor='white'
)
plot.show()

In [43]:
### sumar 1-2, 3-4, 8-9, 10-11
plot = go.Figure()
best_params = [0.1, 0.1, 0.25]
df = method_df_list[0].copy()

replace_dict = { "1": 0, "2": 0, "3": 1, "4": 1, "8": 2, "9": 2, "10": 3, "11": 3 }

for ind, df in enumerate(method_df_list): 
    ### proceso agregar mismo sujeto ###
    df = df[df["threshold"] == best_params[0]]
    df = df.replace({"Unnamed: 0": replace_dict})
    df["subject"] = "Sujeto " + df["Unnamed: 0"].astype("str")
    df = df.drop("Unnamed: 0", axis=1)
    df = df.groupby("subject")[["true_positives", "false_positives", "false_negatives"]].sum()
    df["recall"] = df["true_positives"] / (df["true_positives"] + df["false_negatives"])
    df["precision"] = df["true_positives"] / (df["true_positives"] + df["false_positives"]) 
    df["f1-score"] = (2 * df["recall"] * df["precision"]) / (df["recall"] + df["precision"])

    plot.add_traces([
        go.Bar(x=df.index, y=df["f1-score"], name=f"Método {ind+1}", marker_color=color_dict[ind+1])
    ])

plot.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='rgb(24, 24, 24)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
        title={
            "text": "<b>Threshold</b>",
            'font': {
                'size': 14,
            }
        },
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        linecolor='rgb(24, 24, 24)',
        linewidth=2,
        title={
            "text": "<b>F1-score</b>",
            'font': {
                'size': 14,
            }
        }
    ),
    title={
        'text': "Rendimiento de los métodos según el sujeto a analizar",
        'font': {
            'size': 18,
        },
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'bordercolor': '#000000',
        'borderwidth': 0,
    },
    width=1000,
    height=500,
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    template="plotly_white",
    showlegend=True,
    plot_bgcolor='white'
)
plot.show()



### Análisis del experimento 2.2

In [68]:
experiment2_df = (
    pd.read_csv("experiment2_dynamic2.csv").fillna(0)
)
experiment2_df = experiment2_df[experiment2_df["Unnamed: 0"] != "total"]
experiment2_df.head()

,Unnamed: 0,num_real_blinks,num_method_blinks,num_real_closed_eyes,num_method_closed_eyes,num_hit_closed_eyes,true_positives,false_positives,false_negatives,accuracy,precision,recall,f1-score,method,threshold_perc,alpha_val
1,1,38,126,315,1357,210,37,89,1,0.973684,0.293651,0.973684,0.451220,1,0.2,0
2,10,72,379,574,3019,441,71,308,1,0.986111,0.187335,0.986111,0.314856,1,0.2,0
3,11,43,183,345,1022,259,41,142,2,0.953488,0.224044,0.953488,0.362832,1,0.2,0
4,2,88,106,1024,1028,816,87,19,1,0.988636,0.820755,0.988636,0.896907,1,0.2,0
5,3,65,87,473,424,279,62,25,3,0.953846,0.712644,0.953846,0.815789,1,0.2,0


In [69]:
method_list = list(experiment2_df["method"].unique())
method_df_list = []
for method in method_list:
    method_df_list.append(experiment2_df[experiment2_df["method"] == method])

In [63]:
plot = go.Figure()

color_dict = {
    1: "#003f5c",
    2: "#bc5090",
    3: "#ffa600",
}

# for ind, df in enumerate(method_df_list): 
df = method_df_list[2].copy()
df = df[df["f1-score"] > 0.9]

x = df["threshold_perc"]
y = df["alpha_val"]
z = df["f1-score"]

X, Y = np.meshgrid(x, y)
Z = griddata((x,y), z, (X, Y), method='cubic')

plot.add_traces([
    go.Contour(x=x, y=y, z=z)
])
plot.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='rgb(24, 24, 24)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
        title={
            "text": "<b>Threshold perc</b>",
            'font': {
                'size': 14,
            }
        },
        dtick=0.05,
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        linecolor='rgb(24, 24, 24)',
        linewidth=2,
        title={
            "text": "<b>alpha-val</b>",
            'font': {
                'size': 14,
            }
        }
    ),
    # title={
    #     'text': "<b>Evolución de la f1-score según el porcentaje del threshold y el parámetro de suavizado</b>",
    #     'font': {
    #         'size': 16,
    #     },
    #     'y': 0.9,
    #     'x': 0.5,
    #     'xanchor': 'center',
    #     'yanchor': 'top'
    # },
    legend={
        'bordercolor': '#000000',
        'borderwidth': 0,
        'title': 'f1-score',
    },
    width=600,
    height=500,
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    template="plotly_white",
    showlegend=True,
    plot_bgcolor='white'
)
plot.show()

In [71]:
### sumar 1-2, 3-4, 8-9, 10-11
plot = go.Figure()
best_params_0 = [0.7, 0.7, 0.65]
best_params_1 = [0.6, 0.45, 0.45]
df = method_df_list[0].copy()

replace_dict = { "1": 0, "2": 0, "3": 1, "4": 1, "8": 2, "9": 2, "10": 3, "11": 3 }

for ind, df in enumerate(method_df_list): 
    ### proceso agregar mismo sujeto ###
    df = df[df["threshold_perc"] == best_params_0[ind]]
    df = df[df["alpha_val"] == best_params_1[ind]]
    df = df.replace({"Unnamed: 0": replace_dict})
    df["subject"] = "Sujeto " + df["Unnamed: 0"].astype("str")
    df = df.drop("Unnamed: 0", axis=1)
    df = df.groupby("subject")[["true_positives", "false_positives", "false_negatives"]].sum()
    df["recall"] = df["true_positives"] / (df["true_positives"] + df["false_negatives"])
    df["precision"] = df["true_positives"] / (df["true_positives"] + df["false_positives"]) 
    df["f1-score"] = (2 * df["recall"] * df["precision"]) / (df["recall"] + df["precision"])

    plot.add_traces([
        go.Bar(x=df.index, y=df["f1-score"], name=f"Método {ind+1}", marker_color=color_dict[ind+1])
    ])

plot.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='rgb(24, 24, 24)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
        title={
            "text": "<b>Threshold</b>",
            'font': {
                'size': 14,
            }
        },
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        linecolor='rgb(24, 24, 24)',
        linewidth=2,
        title={
            "text": "<b>F1-score</b>",
            'font': {
                'size': 14,
            }
        }
    ),
    title={
        'text': "Rendimiento de los métodos según el sujeto a analizar",
        'font': {
            'size': 18,
        },
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    legend={
        'bordercolor': '#000000',
        'borderwidth': 0,
    },
    width=1000,
    height=500,
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    template="plotly_white",
    showlegend=True,
    plot_bgcolor='white'
)
plot.show()

### Análisis del experimento 1: Detección del estado del ojo sobre imágenes (DATASET CEW)

In [3]:
experiment1_df = pd.read_csv("experiment1.csv").drop("Unnamed: 0", axis=1)
experiment1_df.head()

,num_real_blinks,num_method_blinks,num_real_closed_eyes,num_method_closed_eyes,num_hit_closed_eyes,true_positives,false_positives,false_negatives,accuracy,precision,recall,f1-score,method,threshold
0,408,32,3714,59,59,31,1,377,0.075980,0.96875,0.075980,0.140909,1,0.0
1,38,1,315,3,3,1,0,37,0.026316,1.00000,0.026316,0.051282,1,0.0
2,72,9,574,12,12,9,0,63,0.125000,1.00000,0.125000,0.222222,1,0.0
3,43,3,345,6,6,3,0,40,0.069767,1.00000,0.069767,0.130435,1,0.0
4,88,3,1024,6,6,3,0,85,0.034091,1.00000,0.034091,0.065934,1,0.0


#### Separación de los datos por método

In [4]:
method_list = list(experiment1_df["method"].unique())
method_df_list = []
for method in method_list:
    method_df_list.append(experiment1_df[experiment1_df["method"] == method])

#### Plot de todos los resultados por método

In [49]:
df = method_df_list[0].copy()
plot = go.Figure()
plot.add_traces([
    go.Surface(z=df["d2"], x=df["threshold_perc"], y=df["alpha_val"])
])
plot.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=True,
        showticklabels=True,
        linecolor='rgb(24, 24, 24)',
        linewidth=2,
        ticks='outside',
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=True,
        showline=True,
        showticklabels=True,
        linecolor='rgb(24, 24, 24)',
        linewidth=2,
    ),
    width=1000,
    height=500,
    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=True,
    plot_bgcolor='white'
)
plot.show()

KeyError: 'threshold_perc'

##### CONCLUSION: El threshold que optimiza accuracy y f1-score para el método 1 es Th=0.15

In [18]:
method2_df = method_df_list[1].copy()
accuracy_plot = px.line(method2_df, x="threshold", y="accuracy", title="METODO 2: ACCURACY X THRESHOLD")
accuracy_plot.show()
f1_score_plot = px.line(method2_df, x="threshold", y="f1-score", title="METODO 2: F1-SCORE X THRESHOLD")
f1_score_plot.show()

##### CONCLUSION: El threshold que optimiza accuracy y f1-score para el método 2 es Th=0.2

In [21]:
method3_df = method_df_list[2].copy()
accuracy_plot = px.line(method3_df, x="threshold", y="accuracy", title="METODO 3: ACCURACY X THRESHOLD")
accuracy_plot.show()
f1_score_plot = px.line(method3_df, x="threshold", y="f1-score", title="METODO 3: F1-SCORE X THRESHOLD")
f1_score_plot.show()

##### CONCLUSION: El threshold que optimiza accuracy y f1-score para el método 3 es Th=0.4